## User Input

In [1]:
ROUNDOFF = 3
DRIVE_TIME_START = 4
DRIVE_TIME_END = 15
NUM_STORES_START = 1
NUM_STORES_END = 30
COUNTRY_CODE = 'SG'

## Setting up the environ vars and GCP connections

In [2]:
import os
os.chdir('..')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../cred/credentials.json'

In [3]:
%load_ext autoreload
%autoreload 2

# call methods for created packages

from optimal_location.metrics_kpi import get_kpi_metrics

from optimal_location.utils import *
from optimal_location.aoi import *

from ml_package.gcp_conn import *
from ml_package.bigquery_conn import *

import warnings
warnings.filterwarnings('ignore')

In [4]:
# call methods from existing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import calendar
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

sns.set({'figure.figsize' : (15,8)})

In [5]:
## call modeling methods

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,davies_bouldin_score,calinski_harabasz_score
from scipy.spatial.distance import cdist


In [6]:
## Queries
def fetch_country_city_code(country_code):
    sql_query = f"""
    
    select 
        distinct(v.global_entity_id),
        g.country_name,
        g.country_code,
        v.location.city as city_name,
        v.location.city_id 
    from `fulfillment-dwh-production.curated_data_shared_central_dwh.vendors` v
    join 
    `fulfillment-dwh-production.curated_data_shared_central_dwh.global_entities` g
    on v.global_entity_id = g.global_entity_id
    where v.location.city_id is not NULL and
    g.country_code = '{country_code}'
    
    """
    
    return pd.read_gbq(sql_query)

def get_order_info(country_code,city_id,input_params):
    
    if len(city_id)==1:
        city_id = city_id*2
    
    query = f"""
    SELECT store_type_l2,
        ROUND(delivery_location.latitude,{input_params['roundoff']}) lat,
        ROUND(delivery_location.longitude, {input_params['roundoff']}) long,
        o.global_entity_id,
        v.location.city_id as city_id,
        COUNT(distinct analytical_customer_id) num_customers,
        COUNT(1) num_orders,
        SUM(i.quantity) num_items,
        SUM(value.gmv_local) gmv
        FROM `fulfillment-dwh-production.curated_data_shared_central_dwh.orders` o, UNNEST(items) AS i
        LEFT JOIN `fulfillment-dwh-production.curated_data_shared_central_dwh.vendors` v USING(vendor_id)
        WHERE o.global_entity_id in 
                    (select global_entity_id 
                    from `fulfillment-dwh-production.curated_data_shared_central_dwh.global_entities` 
                    where country_code = '{country_code}' )
        AND v.location.city_id in {city_id}
        AND DATE(placed_at) BETWEEN '{input_params['start_date']}' AND '{input_params['end_date']}'
        AND order_status = 'sent'
        AND o.is_own_delivery
        AND store_type_l2 in ('restaurants')
        AND delivery_location.latitude IS NOT NULL
        AND delivery_location.longitude IS NOT NULL
        GROUP BY 1, 2, 3,4,5
      
    """
    return pd.read_gbq(query)

def get_order_info_with_city_boundaries(country_code,city_id,input_params):

    new_query= f"""
    WITH polygon_boundaries AS
    (
    SELECT
    store_type_l2,
    lat,
    long,
    global_entity_id,
    city_id,
    num_customers,
    num_orders,
    num_items,
    gmv,
    ST_CONTAINS(
    st_geogfromgeojson(
            (
                select 
                polygons_geojson
                from `fulfillment-dwh-production.curated_data_shared_location.geolocator_cities`
                where country_iso_a2 = '{country_code}' 
                and platform_city_id = '{city_id}'
            )
    ),
    ST_GEOGPOINT(long, lat)
    ) AS `contains`
    FROM (
            SELECT 
            store_type_l2,
            ROUND(delivery_location.latitude,3) lat,
            ROUND(delivery_location.longitude, 3) long,
            o.global_entity_id,
            v.location.city_id as city_id,
            COUNT(distinct analytical_customer_id) num_customers,
            COUNT(1) num_orders,
            SUM(i.quantity) num_items,
            SUM(value.gmv_local) gmv
            FROM `fulfillment-dwh-production.curated_data_shared_central_dwh.orders` o, UNNEST(items) AS i
            LEFT JOIN `fulfillment-dwh-production.curated_data_shared_central_dwh.vendors` v USING(vendor_id)
            WHERE o.global_entity_id in 
                        (select global_entity_id 
                        from `fulfillment-dwh-production.curated_data_shared_central_dwh.global_entities` 
                        where country_code = '{country_code}'  )
            AND v.location.city_id = {city_id}
            AND DATE(placed_at) BETWEEN '{input_params['start_date']}' AND '{input_params['end_date']}'
            AND order_status = 'sent'
            AND o.is_own_delivery
            AND store_type_l2 in ('restaurants')
            AND delivery_location.latitude IS NOT NULL
            AND delivery_location.longitude IS NOT NULL
            GROUP BY 1, 2, 3,4,5

    )
    )
    select * 
    from polygon_boundaries 
    where polygon_boundaries.contains = true
    """
    return pd.read_gbq(new_query)

In [7]:
#Some helper methods to load unload the data

def get_km(k,x,y):
    
    kmeans = KMeans(n_clusters=k,random_state = 0 , max_iter=10)
    wt_kmeansclus = kmeans.fit(x, sample_weight = y)
    return wt_kmeansclus

def get_elbow_score(wt_kmeansclus,X):

    distortion = sum(np.min(cdist(X, wt_kmeansclus.cluster_centers_,
                                    'euclidean'), axis=1)) / X.shape[0]
    inertia = wt_kmeansclus.inertia_

    return distortion, inertia

def get_cluster_centres(centers):
    
    centers_with_id = '{'\
                    + ','.join([f"{id_ + 1} : {tuple(center)}" for id_,center in enumerate(centers)])\
                    + '}'
    return centers_with_id 


def get_score(k, X, y):
    
    if k is not 1:
    
        km = get_km(k,X,y)
        y_pred = km.predict(X,sample_weight = y)

        sil = silhouette_score(X, y_pred)
        mapping1,mapping2 = get_elbow_score(km,X)
        centers = get_cluster_centres(km.cluster_centers_)
    
    else:
        cluster_center = get_single_cluster_centroid(X,y)
        centers = get_cluster_centres(cluster_center)
        sil, mapping1, mapping2 = 0, 0, 0

    return {'cluster_no': k, 
            'silhoutte_score': sil,
            'elbow_score_distortion': mapping1,
            'elbow_score_inertia': mapping2,
            'centroids': centers
           }


def get_order_info_within_city_boundaries(bq_conn,input_params):
    
    queried_data = bq_conn.get_customer_order_data(input_params["country_code"],
                                                   input_params['start_date'],
                                                   input_params['end_date'],
                                                   input_params['roundoff'])
    
    return queried_data



def get_aoi(order_info,warehouse_polygon_df,extraction_params):
    
    existing_coverage = get_existing_coverage(extraction_params,warehouse_polygon_df,order_info)
    aoi = generate_aoi(order_info,existing_coverage)
    return aoi

def get_model_input(aoi):
    
    grouped = aoi.groupby(by=['lat', 'long']).sum().reset_index()
    locations = grouped[['lat', 'long']]
    weight = grouped['num_items']
    return locations, weight

def preprocess_data(order_info):
    order_info = order_info[
        (order_info['num_customers']>0)
    ]
    return order_info

def get_order_date_range(delta_days):
    obj = date.today() + relativedelta(months=-1)
    last_month = obj.month
    last_month_yr = obj.year
    last_month_dt = calendar.monthrange(date.today().year, last_month)[-1]
    from_date = date(last_month_yr, last_month, last_month_dt) - timedelta(days=delta_days)
    return str(from_date), str(date(last_month_yr, last_month, last_month_dt))

def get_single_cluster_centroid(locations,weight):
    weight = weight.to_frame()
    
    lat = (weight['num_items'] * locations['lat']).sum()/weight['num_items'].sum()
    long = (weight['num_items'] * locations['long']).sum()/weight['num_items'].sum()
    return np.array([[lat,long]])

def get_filename(input_params,city_id):
    filename = f"kmeans_centroids_{input_params['country_code']}_{city_id}_cluster_{min(input_params['num_stores'])}"\
    f"-{max(input_params['num_stores'])}_drivetime_{min(input_params['drive_time'])}"\
    f"-{max(input_params['drive_time'])}_{get_current_time()}.csv"
    return filename

def get_polygon_path(city_id,input_params):
    
    polygon_file = f"dmart_polygon_{input_params['country_code']}_{city_id}\
    _{min(input_params['drive_time'])}-{max(input_params['drive_time'])}_latest.json".replace(' ','')

    folder_name = 'existing_dmart_bucket'
    path_for_polygon = f'gs://{bucket_name}/{folder_name}/{polygon_file}'
    return path_for_polygon

def get_polygon_df(path_for_polygon):
     
    json_ = get_polygons_for_locations(path_for_polygon)
    warehouse_polygon_df = convert_json_to_dataframe(json_)
    return warehouse_polygon_df

## Bucket Name

In [8]:
project_id = 'dh-darkstores-stg'
bucket_name = "qc-store-growth-expansion-stg"

## Path to fetch polygon json

## Path to save cluster centers

In [9]:
bucket_name = "qc-store-growth-expansion-stg"
project_name = 'optimal_location_bucket'
path_to_save = f'gs://{bucket_name}/{project_name}'
path_to_save

'gs://qc-store-growth-expansion-stg/optimal_location_bucket'

In [10]:
GCSOperations("qc-store-growth-expansion-stg")
bq_conn = BigQueryOperations(project_id)

In [11]:
country_city_code = fetch_country_city_code(COUNTRY_CODE)
city_id = tuple(country_city_code['city_id'].drop_duplicates().to_list())

In [12]:
country_city_code

,global_entity_id,country_name,country_code,city_name,city_id
0,FP_SG,Singapore,SG,singapore,1
1,FP_SG,Singapore,SG,Singapore,1


In [13]:
# get_order_info_within_city_boundaries(*input_params)

## Input parameters to trigger the model creation

In [14]:
input_params={
    'country_code':COUNTRY_CODE,
    'city_ids': city_id,
    'num_stores':range(NUM_STORES_START ,NUM_STORES_END + 1),
    'drive_time':range(DRIVE_TIME_START, DRIVE_TIME_END + 1),
    'drive_mode':['driving','cycling'],
    'roundoff': ROUNDOFF
}

In [15]:
input_params['start_date'],input_params['end_date'] = get_order_date_range(delta_days = 365)

In [16]:
input_params

{'country_code': 'SG',
 'city_ids': (1,),
 'num_stores': range(1, 31),
 'drive_time': range(4, 16),
 'drive_mode': ['driving', 'cycling'],
 'roundoff': 3,
 'start_date': '2020-12-31',
 'end_date': '2021-12-31'}

In [17]:
# %%time

# order_info = get_order_info_within_city_boundaries(bq_conn,input_params)

In [18]:
%%time
all_cities_order_info = get_order_info(COUNTRY_CODE,city_id,input_params)

CPU times: user 684 ms, sys: 92.9 ms, total: 777 ms
Wall time: 28.8 s


In [19]:
all_cities_order_info['city_id'].value_counts()

1    24941
Name: city_id, dtype: int64

In [20]:
all_cities_order_info = preprocess_data(all_cities_order_info)

In [21]:
%%time
for city_id in input_params['city_ids']:
    print(f'city_id : {city_id}')
    print('-'*20)
    
    order_info = all_cities_order_info[all_cities_order_info['city_id']==city_id]
    
    path_for_polygon = get_polygon_path(city_id,input_params)
    warehouse_polygon_df = get_polygon_df(path_for_polygon)
    print(f"polygons fetched from {path_for_polygon}")
    
    existing_num_dmarts = warehouse_polygon_df[['latitude','longitude']].drop_duplicates().shape[0]
    
    
    notebook_output = pd.DataFrame()
    
        
    for vehicle_type in input_params['drive_mode']:
        print(f'vehicle_type : {vehicle_type}')
        print('-'*20)
        model_output = dict()
        
        for drive_time in input_params['drive_time']:
            


            print(f'drive_time :{drive_time}')
                
            extraction_params={
                                    'driving_time':drive_time,
                                    'driving_mode':vehicle_type
                                }
            try:
                aoi = get_aoi(order_info,warehouse_polygon_df,extraction_params)
                print(f'aoi shape {aoi.shape}')
                
                
                for cluster_no in input_params['num_stores']:
                    
                    if not cluster_no%5 :
                        print(f'cluster_no : {cluster_no}')

                    model_output['num_clusters'] = cluster_no
                    model_output['vehicle_type'] = vehicle_type
                    model_output['model_run_date'] = get_current_time()
                    model_output['delivery_time'] = drive_time



                    if aoi.shape[0] <= cluster_no:
                        print(f'cluster_no : {cluster_no}')
                        print("Number of locations less than requested stores")
                        continue

                    else:

                        locations,weight = get_model_input(aoi)

                        model_output['cluster_array'] = get_score(cluster_no,locations,weight)['centroids']
                        notebook_output = notebook_output.append(
                                            pd.DataFrame(model_output,index = [city_id])
                                            )
            except Exception as error:
                print(f"error_message : {error}")
    print('='*20)
    
    if not notebook_output.empty:
        notebook_output.reset_index(inplace = True)
        notebook_output.rename(columns={'index':'dwh_city_id'},inplace = True)
        notebook_output['country_code'] = COUNTRY_CODE
        notebook_output['existing_dmarts'] = existing_num_dmarts
        filename = get_filename(input_params,city_id)
        notebook_output.to_csv(f'{path_to_save}/{filename}')
        print(f'File Saved at {path_to_save}/{filename}')
    else:
        continue

city_id : 1
--------------------
polygons fetched from gs://qc-store-growth-expansion-stg/existing_dmart_bucket/dmart_polygon_SG_1_4-15_latest.json
vehicle_type : driving
--------------------
drive_time :4
error_message : 'lat_long'
drive_time :5
aoi shape (21290, 9)
cluster_no : 5
cluster_no : 10
cluster_no : 15
cluster_no : 20
cluster_no : 25
cluster_no : 30
drive_time :6
error_message : 'lat_long'
drive_time :7
aoi shape (16517, 9)
cluster_no : 5
cluster_no : 10
cluster_no : 15
cluster_no : 20
cluster_no : 25
cluster_no : 30
drive_time :8
error_message : 'lat_long'
drive_time :9
error_message : 'lat_long'
drive_time :10
aoi shape (7329, 9)
cluster_no : 5
cluster_no : 10
cluster_no : 15
cluster_no : 20
cluster_no : 25
cluster_no : 30
drive_time :11
error_message : 'lat_long'
drive_time :12
error_message : 'lat_long'
drive_time :13
error_message : 'lat_long'
drive_time :14
error_message : 'lat_long'
drive_time :15
aoi shape (693, 9)
cluster_no : 5
cluster_no : 10
cluster_no : 15
clust

In [22]:
notebook_output

,dwh_city_id,num_clusters,vehicle_type,model_run_date,delivery_time,cluster_array,country_code,existing_dmarts
0,1,1,driving,2022-1-4_10-17,5,"{1 : (1.3612927090971114, 103.84605594970736)}",SG,15
1,1,2,driving,2022-1-4_10-17,5,"{1 : (1.3463533737473352, 103.89192591682533),...",SG,15
2,1,3,driving,2022-1-4_10-17,5,"{1 : (1.4334505562716047, 103.81738132806638),...",SG,15
3,1,4,driving,2022-1-4_10-18,5,"{1 : (1.3184034085303415, 103.85064883344637),...",SG,15
4,1,5,driving,2022-1-4_10-18,5,"{1 : (1.3127289060416054, 103.8477164614365),2...",SG,15
...,...,...,...,...,...,...,...,...
235,1,26,cycling,2022-1-4_10-28,15,"{1 : (1.4018635861988846, 103.74653481862802),...",SG,15
236,1,27,cycling,2022-1-4_10-28,15,"{1 : (1.3318280595364032, 103.90565872033389),...",SG,15
237,1,28,cycling,2022-1-4_10-28,15,"{1 : (1.3295808654563164, 103.79471285249045),...",SG,15
238,1,29,cycling,2022-1-4_10-28,15,"{1 : (1.3797141524722552, 103.8889666281463),2...",SG,15
